In [2]:
import pandas as pd
import json

from serlo.mysql import db

In [3]:
class entity:
    def __init__(self, id, empty_boxes):
        self.id = id
        self.empty_boxes = empty_boxes
        
    

In [4]:
#def get_items(obj):

def get_keys(obj, stack):
    #print("this: ")
    #print(obj)
    for k, v in obj.items():
      k2 = ([k] if k else []) + stack # don't return empty keys
      if v and isinstance(v, dict):
        for c in get_keys(v, k2):
          yield c
      else: # leaf
        yield k2

def get_values(obj):
    for v in obj.values():
      if not v: continue
      if isinstance(v, dict):
        for c in get_values(v):
          yield c
      else: # leaf
        yield v if isinstance(v, list) else [v]
        
def get_value_from_key_array(obj, keys):
    #print(keys)
    if len(keys) == 0:
        return
    if len(keys) == 1:
        return obj[keys[0]]
    else:
        key = keys.pop(-1)
        #print(key)
        return get_value_from_key_array(obj[key], keys)

  #return list(get_keys(obj,[])), list(get_values(obj))

In [5]:
def find_empty_boxes(obj):
    empty_boxes = []
    #print(obj)
    keys = list(get_keys(obj, []))
    #print("keys")
    #print(keys)
    for i in range(len(keys)):
        leaf = keys[i][0]
        if leaf == "plugin":
            plugin = {}
            if len(keys[i]) == 1:
                plugin = obj
            else:
                #print("test")
                plugin = get_value_from_key_array(obj, keys[i][1:])
            if plugin["plugin"] == "box":
                if is_empty_box(plugin["state"]):
                    empty_boxes.append(plugin["state"]["anchorId"])
                    #print(1)
        elif leaf == "state":
            #print(i)
            #print(keys[i])
            #print("value")
            #print(get_value_from_key_array(obj, keys[i]))
            state = get_value_from_key_array(obj, keys[i])
            if type(state) == list:
                for sub_obj in state:
                    #print("this: ")
                    #print(sub_obj)
                    #print("theses: ")
                    #print(find_empty_boxes(sub_obj))
                    empty_boxes.extend(find_empty_boxes(sub_obj))
    return empty_boxes
                    
def handle_boxes(boxes):
    if boxes:
        for box in boxes:
            if is_empty_box(box):
                handle_empty_box(box)
                    
                    
def is_empty_box(box):
    return is_empty_object(box["content"])

def is_empty_object(obj):
    match obj["plugin"]:
        case "rows":
            for row in obj["state"]:
                if !is_empty_object(obj["state"]):
                    return False
        case "text":
            for text in obj["state"]:
                if text["type"] == 'p':
                    return is_empty_text(text["children"])
                else:
                    print("unhandled text type: ")
                    print(text["type"])
                    return False
        case _:
            print("unhandled plugin: ")
            print(obj["plugin"])
            return False
            
    return True
    
def is_empty_text(text):
    for child in text["children"]:
        if child["text"]:
            return True
    return False

SyntaxError: invalid syntax (420016008.py, line 49)

In [6]:
df = pd.read_sql("""
SELECT entity.id, value
FROM entity_revision_field, entity
where field = "content"
    AND entity.current_revision_id = entity_revision_field.entity_revision_id;
""", db)

df.head()

/home/moe/.asdf/installs/python/3.10.1/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,value
0,1495,"{""plugin"":""article"",""state"":{""introduction"":{""..."
1,1497,"{""plugin"":""article"",""state"":{""introduction"":{""..."
2,1499,"{""plugin"":""article"",""state"":{""introduction"":{""..."
3,1501,"{""plugin"":""article"",""state"":{""introduction"":{""..."
4,1503,"{""plugin"":""article"",""state"":{""introduction"":{""..."


In [7]:
entities = []
for i, revision in df.iterrows():
    print("id")
    print(revision["id"])
    try:
        currentObject = json.loads(revision["value"])
        if isinstance(currentObject, dict):
            empty_boxes = find_empty_boxes(currentObject)
            if len(empty_boxes):
                entities.extend(entity(revision["id"], empty_boxes))
    except:
        print(revision["value"])
            
entities

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[]

In [18]:
#df["id" == 16078]
df[df.id == 16078]

,id,value
6985,16078,http://www.youtube.com/watch?v=J2IO2sBJj-4


In [19]:
bytes(df["value"][400], "latin1").decode("utf8")

'{"plugin":"solution","state":{"prerequisite":{"id":"1827","title":"Flächenberechnung"},"strategy":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"steps":{"plugin":"rows","state":[{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Im Bereich "},{"type":"math","src":"x=1","inline":true,"children":[{"text":"x=1"}]},{"text":" bis "},{"type":"math","src":"x=2","inline":true,"children":[{"text":"x=2"}]},{"text":" ist die Funktion "},{"type":"math","src":"f(x)=x^3","inline":true,"children":[{"text":"f(x)=x^3"}]},{"text":" positiv und hat keine Nullstellen, so dass es reicht, das Integral in den angegebenen Grenzen zu berechnen."}]}]},{"plugin":"text","state":[{"type":"p","children":[{"text":""},{"type":"math","src":"f(x)=x^3","inline":true,"children":[{"text":"f(x)=x^3"}]},{"text":" , "},{"type":"math","src":"a=1","inline":true,"children":[{"text":"a=1"}]},{"text":" , "},{"type":"math","src":"b=2","inline":true,"children":[{"text":"b=2"}]},{"text":

In [14]:
test = '{"plugin":"article","state":{"introduction":{"plugin":"articleIntroduction","state":{"explanation":{"plugin":"text","state":[{"type":"p","children":[{"text":"In diesem Artikel werden Boxen und ihre Verwandten ausprobiert :)"}]}]},"multimedia":{"plugin":"image","state":{"src":"","caption":{"plugin":"text","state":[{"type":"p","children":[{}]}]}}},"illustrating":true,"width":50}},"content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Text Anfang"}]}]},{"plugin":"box","state":{"title":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das hier ist ein Merksatz. In der Plugin-Beschreibung steht auch, sodass dieser bisher wohl nicht besonders semantisch verwendet wurde."}]},{"type":"p","children":[{"text":""}]}]}]},"type":"blank","anchorId":"box80970"}},{"plugin":"text","state":[{"type":"p","children":[{"text":"Zwischentext zwischen den Elementen"}]}]},{"plugin":"box","state":{"title":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das hier ist ein Zitat. Angeblich ist es einfach eingerückt, oder passiert hier noch mehr?"}]},{"type":"p","children":[{"text":""}]}]}]},"type":"quote","anchorId":"box35521"}},{"plugin":"text","state":[{"type":"p","children":[{"text":"Noch ein schicker Zwischentext"}]}]},{"plugin":"spoiler","state":{"title":"Das ist der Spoiler-Titel","content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das ist der Spoiler-Inhalt, hoffentlich."}]}]}]}}},{"plugin":"text","state":[{"type":"p","children":[{"text":"Und zum Abschluss kommt noch mal normaler Text"}]}]},{"plugin":"text","state":[{"type":"p","children":[{"text":"Und hier ein Experiment mit Tastendrücken: "},{"type":"math","src":"\\\\fbox {~1~} ~\\\\fbox{Enter}~\\\\fbox{~2~} ~\\\\fbox{Enter}~\\\\fbox{~+~}","inline":true,"children":[{"text":"1 Enter"}]},{"text":""}]}]},{"plugin":"text","state":[{"type":"p","children":[{}]}]},{"plugin":"box","state":{"type":"note","title":{"plugin":"text","state":[{"type":"p","children":[{}]}]},"anchorId":"box32917","content":{"plugin":"rows","state":[{"plugin":"text","state":[{"type":"p","children":[{"text":"Das ist eine schicke neue Box."}]}]}]}}}]},"exercises":[],"exerciseFolder":{"id":"","title":""},"relatedContent":{"articles":[],"courses":[],"videos":[]},"sources":[]}}'

In [21]:
obj = json.loads(test)
keys = list(get_keys(obj, []))
keys

[['plugin'],
 ['plugin', 'introduction', 'state'],
 ['plugin', 'explanation', 'state', 'introduction', 'state'],
 ['state', 'explanation', 'state', 'introduction', 'state'],
 ['plugin', 'multimedia', 'state', 'introduction', 'state'],
 ['src', 'state', 'multimedia', 'state', 'introduction', 'state'],
 ['plugin',
  'caption',
  'state',
  'multimedia',
  'state',
  'introduction',
  'state'],
 ['state', 'caption', 'state', 'multimedia', 'state', 'introduction', 'state'],
 ['illustrating', 'state', 'introduction', 'state'],
 ['width', 'state', 'introduction', 'state'],
 ['plugin', 'content', 'state'],
 ['state', 'content', 'state'],
 ['exercises', 'state'],
 ['id', 'exerciseFolder', 'state'],
 ['title', 'exerciseFolder', 'state'],
 ['articles', 'relatedContent', 'state'],
 ['courses', 'relatedContent', 'state'],
 ['videos', 'relatedContent', 'state'],
 ['sources', 'state']]

In [22]:
key_arr = ['plugin', 'introduction', 'state']
value = get_value_from_key_array(obj, key_arr)
value

'articleIntroduction'